In [5]:
from gsheets import Sheets
import pandas as pd
import numpy as np
import datetime as dt
import time
# import filepaths
# import content_feedback_report
# import schedule_methods
# import lexi_cobra_sync
import pygsheets
import pandas as pd
import numpy as np
import datetime as dt
import time
# import schedule_methods
import math
import datetime
from datetime import timedelta  
import emoji
import re
import warnings
import statistics
from datetime import date
import matplotlib.pyplot as plt
import seaborn as sns
import infrastructure 
from calendar import monthrange
from dateutil.relativedelta import relativedelta
import xlsxwriter
import send_email
from colorama import Fore, Style
import filepath
import smartsheet

In [6]:
# Smartsheet API token
key = 'gMX4xu48SUIH7rMEj4iaj7rIsQJ2ERbmgiYMJ'
key = 'wf3Jr9OaAT5OE8gE8yctPqfmzrv6dUFATUODD'
# SMS Creative Submission sheet ID
# sheet_id = 2525300306956164 #FAKE SHEET
#sheet_id = 8253683024220036 #REAL SHEET
sheet_id = 5426394364333956

# Downloading Lexi, Offers, CW, CT, and Mamba

In [7]:
lexi_original = infrastructure.get_lexi()
lexi = lexi_original.copy()
lexi = infrastructure.transform_sms_df(lexi)
lexi = lexi[lexi['Delivered'] > 0]
# lexi[['Hitpath Offer ID']] = lexi[['Hitpath Offer ID']].apply(pd.to_numeric, errors='coerce')
# lexi.dropna(subset=['Hitpath Offer ID'], inplace = True)

lexi['eCPM'] = lexi['Revenue']*1000 / lexi['Delivered']
lexi['CTR'] = lexi['Clicks'] / lexi['Delivered']
lexi['Hitpath Offer ID'] = lexi['Hitpath Offer ID'].astype(str)
lexi['Hitpath Offer ID'] = lexi['Hitpath Offer ID'].str.replace('.0', '')

/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_20628/3359913376.py:1: DtypeWarning: Columns (0,1,2,3,4,6,8,9,10,11,12,14,16,26,27,28,29,30,31,32,33,34,35,36,37,44,48,51,54,56,58,59,63) have mixed types. Specify dtype option on import or set low_memory=False.
  lexi_original = pd.read_csv(filepath.input_folder + 'SS_LC_merged_data.csv')
/Users/liliguo/Desktop/github/SMS_Report/Daily_Reporting/Creative_package/infrastructure.py:77: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  schedule2['Date']=pd.to_datetime(schedule2[0],errors='coerce')
/Users/liliguo/Desktop/github/SMS_Report/Daily_Reporting/Creative_package/infrastructure.py:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

In [8]:
cw = infrastructure.get_lanina()
cw.dropna(subset=['Reporting Content ID'],inplace=True)
# cw[['OfferIDs']] = cw[['OfferIDs']].apply(pd.to_numeric, errors='coerce')
cw.dropna(subset=['OfferIDs'], inplace = True)
mask = cw['Type (Pitch)'] == 'HOL'
cw = cw[~mask]
cw['Allocation Period (Date Added)'] = pd.to_datetime(cw['Allocation Period (Date Added)'], format='mixed')
cw.dropna(subset=['Reporting Content ID'],inplace=True)
cw['OfferIDs'] = cw['OfferIDs'].astype(str)

In [9]:
offers_original = infrastructure.get_smartsheet('offers_sms')
offers = offers_original.copy()
offers.dropna(subset=['Hitpath Offer ID'],inplace=True)
offers['Hitpath Offer ID'] = offers['Hitpath Offer ID'].astype(int).astype(str)
offers.rename(columns={'Redirect Link (Ops will update when setup)':'Redirect Link'}, inplace=True)
# offers['Hitpath Offer ID'] = offers['Hitpath Offer ID'].astype(int)

In [10]:
ow = infrastructure.get_smartsheet('ow_sms')
ow['Available Shortcode'] = ow['Available Shortcode'].apply(lambda x: x.split('\n'))
ow_status_dict = ow.set_index('Hitpath Offer ID')['Status'].to_dict()
live_ow_shortcodes_dict = ow[ow['Status']=='Live'].set_index('Hitpath Offer ID')['Available Shortcode'].to_dict()
not_live_ows = ow[~(ow['Status']=='Live')]['Hitpath Offer ID'].unique().tolist()
ow_hitpath_list = ow['Hitpath Offer ID'].unique().tolist()

In [11]:
lexi = lexi.sort_values(by='Date')

lexi_30 = lexi[(lexi['Date'].dt.date >= (date.today() + timedelta(days=-30)))].copy()
lexi_90 = lexi[(lexi['Date'].dt.date >= (date.today() + timedelta(days=-90)))].copy()
# Will use this list of hitpaths to consider for content requests--Production offers sending in the past 30 days
eligible_hitpaths = lexi_30[lexi_30['Send Strategy']=='P']['Hitpath Offer ID'].unique()

In [12]:
ct =infrastructure.get_smartsheet('content_sms')
ct['Submission Date'] = pd.to_datetime(ct['Submission Date'])
ct['Date Added to La Nina (Akshad)'].replace(to_replace=r"[a-zA-Z- ()]", value="", regex=True, inplace=True)
ct['Date Added to La Nina (Akshad)'] = pd.to_datetime(ct['Date Added to La Nina (Akshad)'], errors='coerce')

/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_20628/3074324814.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ct['Submission Date'] = pd.to_datetime(ct['Submission Date'])
/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_20628/3074324814.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ct['Date Added to La Nina (Akshad)'] = pd.to_datetime(ct['Date Added to La Nina (Akshad)'], errors='coerce')


In [13]:
mamba = infrastructure.get_mamba()
mamba['Affiliate ID']=mamba['Dataset'].str.split('_').str[2]
mamba[['Limit','Offset']] = mamba[['Limit','Offset']].fillna("")
mamba['Drop Number'] = mamba['Drop'].str.split(expand=True)[1].astype(int)
mamba['Day of Week'] = mamba['Date'].dt.dayofweek
mamba.rename(columns={"Segment ": "Segment"},inplace=True)

/Users/liliguo/Desktop/github/SMS_Report/Daily_Reporting/Creative_package/infrastructure.py:77: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  schedule2['Date']=pd.to_datetime(schedule2[0],errors='coerce')
/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_20628/2744609448.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mamba['Affiliate ID']=mamba['Dataset'].str.split('_').str[2]
/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_20628/2744609448.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

# Functions

In [14]:
def acceptable_performance(row):
    """
    Determines if a given creative meets performance benchmarks for sending

    :param row: a row of a dataframe (Series) that has columns 'CTR', 'CTR_min_target', 'eCPM', and 'eCPM_min_target'
                The dataframe should contain information about a given creative relative to a benchmark
    :return: True if given creative passes benchmarks. False if creative fails benchmarks
    """
    if row['OOR'] >= 0.02:
        return False
    elif row['Num_Drops'] > 2:
        if (row['CTR'] < row['CTR_min_target']) & (row['eCPM'] < row['eCPM_min_target']):
            return False
    elif (row['Shortcode Name'] == 'DSS') and (row['Num_Drops'] > 1):
        if (row['CTR'] < row['CTR_min_target']) & (row['eCPM'] < row['eCPM_min_target']):
            return False
    
    return True

In [15]:
def make_shortcode_ctr_benchmark(row):
    
    # Noticed that DSS was performing worse with CTR, so want to lower the threshold for requesting more content
    if row['Shortcode Name'] == 'DSS':
        benchmark = row['CTR_mean'] - (0.5 * row['CTR_std'])
    else:
        benchmark = row['CTR_mean'] - row['CTR_std']

    return benchmark

In [16]:
def make_stats_by_creative(min_eCPM_target=10):
    """
    
    :param min_eCPM_target: An int of the minimum eCPM a content piece should generate to be considered 'sendworthy'.
                            Default is $10
    :return: A dictionary that has keys of (Shortcode Name, Hitpath Offer ID) and values of ints which are the
            number of creatives per Shortcode/Hitpath ID combo that are bad.
    """
    # Get Num_drops, OOR, CTR, and eCPM stats for each creative
    stats_by_creative = lexi_30.groupby('Creative Type').agg(Optout=('Optout', 'sum'), Clicks=('Clicks', 'sum'), Revenue=('Revenue', 'sum'), Delivered=('Delivered', 'sum'), Shortcode_Name=('Shortcode Name', lambda x: x.value_counts().idxmax()), Hitpath_Offer_ID=('Hitpath Offer ID', lambda x: x.value_counts().idxmax()), 
                                                         Num_Drops=('Date', 'count'))
    stats_by_creative = stats_by_creative.rename(columns={'Shortcode_Name':'Shortcode Name', 'Hitpath_Offer_ID':'Hitpath Offer ID'})

    stats_by_creative['OOR'] = stats_by_creative['Optout'] / stats_by_creative['Delivered']
    stats_by_creative['CTR'] = stats_by_creative['Clicks'] / stats_by_creative['Delivered']
    stats_by_creative['eCPM'] = 1000 * stats_by_creative['Revenue'] / stats_by_creative['Delivered']
    stats_by_creative = stats_by_creative.reset_index()

    # Get 30d stats by shortcode as benchmark to compare to
    stats_by_shortcode_30d = lexi_30.groupby('Shortcode Name').agg(CTR_mean=('CTR', 'mean'), CTR_std=('CTR', 'std')).reset_index()
    stats_by_shortcode_30d['CTR_min_target'] = stats_by_shortcode_30d.apply(make_shortcode_ctr_benchmark, axis=1)
#     stats_by_shortcode_30d['CTR_min_target'] = stats_by_shortcode_30d['CTR_mean'] - stats_by_shortcode_30d['CTR_std']
    stats_by_shortcode_30d = stats_by_shortcode_30d[['Shortcode Name', 'CTR_min_target']]
    stats_by_shortcode_30d['eCPM_min_target'] = min_eCPM_target

    # Merge creative stats with benchmarks
    stats_comparison = stats_by_creative.merge(stats_by_shortcode_30d, how='left', on='Shortcode Name')

    # True if beats benchmarks, else False
    stats_comparison['Sendable'] = stats_comparison.apply(lambda x: acceptable_performance(x), axis=1)

    stats_comparison = stats_comparison[['Creative Type', 'Optout', 'Clicks', 'Revenue', 'Delivered', 'Shortcode Name', 'Hitpath Offer ID', 
                                         'Num_Drops', 'OOR', 'CTR', 'CTR_min_target', 'eCPM', 'eCPM_min_target', 'Sendable']]
    stats_comparison['Hitpath Offer ID'] = stats_comparison['Hitpath Offer ID'].astype(str)

    # Filter for only bad creatives
    bad_creatives_df = stats_comparison[~stats_comparison['Sendable']]
    bad_creatives_df = bad_creatives_df[['Hitpath Offer ID', 'Shortcode Name', 'Creative Type', 'Num_Drops', 'OOR', 'CTR', 'eCPM']]

    # Create dict with keys of (SC Name, Hit ID) and values of number of bad content pieces
    bad_offers_dict = bad_creatives_df.groupby(['Shortcode Name', 'Hitpath Offer ID'])['Creative Type'].count().to_dict()

    return stats_comparison, bad_offers_dict
    

In [17]:
def make_stats_by_offer(hitpath_id_list):
    """
    
    :return:
    """
    # Get Num_drops, OOR, CTR, and eCPM stats for each creative
    stats_by_offer = lexi_30[lexi_30['Hitpath Offer ID'].isin(hitpath_id_list)].groupby(['Hitpath Offer ID']).agg(Optout=('Optout', 'sum'), Clicks=('Clicks', 'sum'), Revenue=('Revenue', 'sum'), Delivered=('Delivered', 'sum'), Num_Drops=('Date', 'count'))

    stats_by_offer['OOR'] = stats_by_offer['Optout'] / stats_by_offer['Delivered']
    stats_by_offer['CTR'] = stats_by_offer['Clicks'] / stats_by_offer['Delivered']
    stats_by_offer['eCPM'] = 1000 * stats_by_offer['Revenue'] / stats_by_offer['Delivered']
    stats_by_offer = stats_by_offer.reset_index()

    # Lower rank is better
    higher_good_ranks = stats_by_offer[['CTR', 'eCPM']].rank(ascending=False)
    higher_bad_ranks = stats_by_offer[['OOR']].rank(ascending=True)

    # Merge ranks onto df
    stats_by_offer = pd.concat([stats_by_offer, higher_good_ranks.add_suffix('_rank')], axis=1)
    stats_by_offer = pd.concat([stats_by_offer, higher_bad_ranks.add_suffix('_rank')], axis=1)


    # Determine priority order based on sum of stat ranks
    stats_by_offer['Priority Order'] = stats_by_offer['OOR_rank'] + stats_by_offer['CTR_rank'] + stats_by_offer['eCPM_rank']
    stats_by_offer['Priority Order'] = stats_by_offer['Priority Order'].rank(ascending=False)
    stats_by_offer = stats_by_offer.sort_values(by='Priority Order')
    stats_by_offer['Hitpath Offer ID'] = stats_by_offer['Hitpath Offer ID'].astype(str)
    stats_by_offer = stats_by_offer.rename(columns={'Num_Drops':'30D Drop Count'})
    priority_dict = stats_by_offer.set_index(['Hitpath Offer ID'])['Priority Order'].to_dict()

    return stats_by_offer, priority_dict

In [18]:
def get_bad_offer_count(row, bad_offers_dict):
    """
    :return: an int of the number of bad creatives for each (Shortcode Name, Hitpath Offer ID) combo
    """
    key = (row['Shortcode Name'], row['Hitpath Offer ID'])
    return bad_offers_dict.get(key, 0)

In [19]:
def find_already_requested_content():
    """
    :return: a list of Hitpath Offer IDs that have had content requested within the past 2 weeks in the 
            SMS Creative Submission smartsheet.
    """
    already_requested_content = ct.copy()
    already_requested_content = already_requested_content[['Submission Date', 'Date Added to La Nina (Akshad)', '# of Pieces Requested Per Shortcode', 'Hitpath Offer ID', 'Request Type (Request)']]
    already_requested_content = already_requested_content[already_requested_content['Request Type (Request)'].str.contains('Content', na=False)]
    already_requested_content = already_requested_content[already_requested_content['# of Pieces Requested Per Shortcode']>0]
    # Any offer without a date in this column has not been added to La Nina, and should not be requested again
    already_requested_content = already_requested_content[((pd.isna(already_requested_content['Date Added to La Nina (Akshad)']))&(already_requested_content['Submission Date']>=pd.Timestamp(date.today()-timedelta(30)))) | 
                                                          (already_requested_content['Date Added to La Nina (Akshad)']>=pd.Timestamp(date.today() - timedelta(21)))]
    # already_requested_content['Shortcode'] = already_requested_content['Shortcode'].apply(lambda x: x.split('\n'))
#     already_requested_content['Date Added to La Nina (Akshad)'] = already_requested_content['Date Added to La Nina (Akshad)'].apply(lambda x: pd.to_numeric(x, errors='coerce'))
#     already_requested_content = already_requested_content.dropna(subset=['Date Added to La Nina (Akshad)'])

    return already_requested_content['Hitpath Offer ID'].unique().tolist()

In [20]:
def business_days_from_date(start_date, n):
    """
    Returns the date of the business day that is n days from the start_date

    :param start_date: a Timestamp of the first date to consider
    :param n: an int of the number of business days ahead to return
    :return: a Timestamp of the business day that is n days ahead of start_date
    """
    current_date = start_date
    while n > 0:
        current_date += timedelta(days=1)
        if current_date.weekday() < 5:  # Check if it's a weekday (Monday to Friday)
            n -= 1
    return current_date

In [21]:
def get_advertiser_emails_dict():
    """
    :return: a dictionary with keys of Advertiser Name and values of a list of
            potential email addresses for that advertiser
    """
    adv_emails = offers_original[(~pd.isna(offers_original['Advertiser'])) & (pd.isna(offers_original['Hitpath Offer ID']))][['Advertiser Rep', 'Advertiser']]
    adv_emails = adv_emails.drop_duplicates(subset='Advertiser', keep='first')
    adv_emails = adv_emails.fillna('')
    adv_emails['Advertiser Rep'] = adv_emails['Advertiser Rep'].apply(lambda x: x.split('\n'))
    adv_emails = adv_emails.set_index('Advertiser')['Advertiser Rep'].to_dict()
    
    return adv_emails

In [22]:
# from fuzzywuzzy import fuzz

# def find_closest_email(name, email_list):
#     email_score = 0
#     closest_email = ''
#     for email in email_list:
#         email_name = email.split('@')[0]
#         similarity_score = fuzz.token_sort_ratio(name, email_name)
#         if similarity_score > email_score: 
#             closest_email = email
    
#     return closest_email

def find_closest_email(name, email_list):
    """
    Takes a name and compares it to a list of email addresses to find the email addresss that
    most closely matches. 

    :param name: a string of a name
    :param email_list: a list of strings of potential emails
    :return: a string of the email that most closely matches the name given.
            If none of the emails make sense, returns a blank string
    """
    # If first and last name are listed, will see if either are in the email address
    for name_part in name.split():
        for email in email_list:
            if not '@' in email:
                continue
            email_name = email.split('@')[0].lower()
            if name_part.lower() in email_name:
                return email
    return ''

In [23]:
def get_advertiser_email_address(row):
    """
    For a given Advertiser/Advertiser Rep combo, finds the email from possible Advertiser emails
    that most closely matches with the Advertiser Rep.

    :param row: a row of a dataframe (Series) with columns of 'Advertiser Rep', and 'Advertiser'
    :return: a string of an email address
    """
    # If it is an OW, there are many different advertisers, so no email is listed
    if row['Hitpath Offer ID'] in ow_hitpath_list:
        return ''
    # Get dict mapping Advertiser to email list
    adv_emails = get_advertiser_emails_dict()
    
    # If Advertiser Rep already has an email listed, use that
    if '@' in row['Advertiser Rep']:
        return row['Advertiser Rep']
    else:
        possible_emails = adv_emails[row['Advertiser']]
        name = row['Advertiser Rep']
        closest_email = find_closest_email(name, possible_emails)

        # If not emails match, use the Advertiser Rep as the email
        if not closest_email:
            closest_email = row['Advertiser Rep']

        # If the Advertiser Rep is blank, use the Advertiser
        if not closest_email:
            closest_email = adv_emails[row['Advertiser']]
        
        # If the Advertiser is blank, leave blank
        if closest_email == ['']:
            closest_email = ''
        
        # If closest email is still a list, use all contents of list, separated by a dash if multiple elements
        if type(closest_email) == list:
            if len(closest_email) == 1:
                closest_email = closest_email[0]
            else:
                closest_email = ' - '.join(closest_email)
    
    return closest_email

In [24]:
def fix_approval_status(status):
    """
    Fix formatting of Custom Creative Allowed and default to Required
    
    :param status: a string from Custom Creative Allowed column in smartsheet
    :return: a string that is formatted the same as seen in SMS Creative Submission with options being
            Required or Not Required. Default to Required
    """
    if pd.isna(status):
        status = 'Required'
    elif status == 'NOT Required':
        status = 'Not Required'
    elif 'Approved' in status:
        status = 'Required'
    return status

In [25]:
def get_starting_ID_number():
    """
    :return: an int of 1 more than the maximum request ID number from the SMS Creative Submission smartsheet
    """
    # Pull number from Request ID
    ID_numbers = [int(re.search(r'\.(\d+)\.', s).group(1)) for s in ct['Request ID']]

    # Starting Request ID is 1 more than maximum
    return max(ID_numbers) + 1

In [26]:
from fuzzywuzzy import fuzz

def find_closest_RX_Rep(name):
    """
    Takes a name and compares it to a list of RX_Reps to find the RX_Rep that
    most closely matches. 

    :param name: a string of a name
    :return: a string of the RX_Rep that most closely matches the name given.
            If none of the RX_Reps make sense, returns a blank string
    """
    if pd.isna(name) or name=='' or name.lower()=='nan':
        return ''
    RX_Rep_list = ['Eamon McMahon', 'Haley Bush', 'Hartley Goode', 'Jennifer Chatellier', 'Jennifer Sporleder', 
                   'Kelly baker', 'Marley Smith', 'Mary Beth Howe', 'Rhiannon Selander', 'Sarah Bowe', 'Surya Hemanth']
    RX_Rep_score = 0
    closest_RX_Rep = ''
    for RX_Rep in RX_Rep_list:
        similarity_score = fuzz.token_sort_ratio(name, RX_Rep)
        if similarity_score > RX_Rep_score: 
            closest_RX_Rep = RX_Rep
            RX_Rep_score = similarity_score
    
    return closest_RX_Rep

/opt/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [27]:
def limit_requests_per_shortcode(requests_per_shortcode, limit=2):
    """Bring any requests per shortcode above the limit down to the limit"""

    if requests_per_shortcode > limit:
        return limit
    else:
        return requests_per_shortcode

In [28]:
def finalize_submission_df(df):
    """
    Properly formats the dataframe containing info on # of Pieces needed to be requested so that it can be pasted into
    the SMS Creative Submission smartsheet. Also, gathers any additional information needed.

    :param df: a dataframe of the # of pieces needed to be requested for each Shortcode/Hitpath ID combo
    :return: a dataframe that is ready to be pasted into the SMS Creative Submission smartsheet
    """
    ct_columns = ['Hitpath ID', 'Request ID', 'High Priority', 'Submission Date',
    'Due Date ', 'Request Sent to Copywriter (Kellie)', 'Unapproved Content Document Link (Copywriter)',
    'Unapproved Jump Page Content (Copywriter)', 'Image ID - Jump Page (Copywriter)', 'Request Type (Request)',
    'Content Sent to Compliance (Kellie)', 'Content Approved by Compliance (Kellie)', 'Date - Jump Page Created (Akshad)',
    'Jump Page Link (Akshad)', 'Creative Approval Status', 'Date - Content sent for approval to network (K.S.)',
    'Date Got Approval from Advertiser (K.S.)', 'Approved Jump Page Link (Kellie)', 'External - Approved Content Document (Kellie)',
    'Ready for Akshad (Kellie Checks; Alert Sent)', 'Date Added to La Nina (Akshad)', 'Offer Type', 'Operational Status (Connected to SMS-OMA)',
    'Hitpath Offer ID', 'Offer Name', 'Vertical', 'Landing Page / Redirect Link', 'Compliance & Additional Notes (Request)',
    '# of Pieces Requested Per Shortcode', '# of Pieces Requested Per Toll Free', 'Shortcode Abbreviations', 'Toll Free Abbreviations', 'Requested By', 
    'RX Rep', 'External AM Email', 'Channel', 'Platform']

    # date_columns = ['Submission Date', 'Due Date ', 'Request Sent to Copywriter (Kellie)',
    # 'Content Sent to Compliance (Kellie)', 'Content Approved by Compliance (Kellie)', 'Date - Jump Page Created (Akshad)',
    # 'Date - Content sent for approval to network (K.S.)',
    # 'Date Got Approval from Advertiser (K.S.)', 'Date Added to La Nina (Akshad)']
    
    # submission_columns = ['Hitpath ID', 'Submission Date', 'Due Date ', 'Request Type (Request)',
    #                   'Creative Approval Status', 'Offer Type', 'Hitpath Offer ID', 'Offer Name',
    #                   'Vertical', 'Landing Page / Redirect Link', 'Compliance & Additional Notes (Request)',
    #                   '# of Pieces Requested Per Shortcode', 'Shortcode', 'Requested By', 'RX Rep',
    #                   'External AM Email', 'Channel', 'Platform']    
    
    df = df.merge(offers[['Hitpath Offer ID', 'Offer Name', 'Custom Creative Allowed', 'Vertical', 'Redirect Link', 'RX Rep', 'Advertiser Rep', 'Advertiser']], how='left', on='Hitpath Offer ID')
    # Merge OW info
    df = df.merge(ow[['Hitpath Offer ID', 'Vertical', 'Offer Wall Link']], how='left', on='Hitpath Offer ID', suffixes=['_normal', '_OW'])
    df['Vertical'] = df.apply(lambda x: x['Vertical_normal'] if pd.isna(x['Vertical_OW']) else x['Vertical_OW'], axis=1)
    df['Landing Page / Redirect Link'] = df.apply(lambda x: x['Redirect Link'] if pd.isna(x['Offer Wall Link']) else x['Offer Wall Link'], axis=1)
    df = df.drop(['Vertical_normal', 'Vertical_OW', 'Offer Wall Link'], axis=1)
    df = df.rename(columns={'Custom Creative Allowed':'Creative Approval Status', 'Num_Pieces_to_Request':'# of Pieces Requested Per Shortcode', 'Shortcode_Names':'Shortcode Abbreviations'})
    
    # df['Creative Approval Status'] = df['Creative Approval Status'].apply(lambda x: fix_approval_status(x))
    df['Advertiser Rep'] = df['Advertiser Rep'].fillna('')
    df['Shortcode Abbreviations'] = df['Shortcode Abbreviations'].apply(lambda x: '\n'.join(x))
    df['External AM Email'] = df.apply(lambda x: get_advertiser_email_address(x), axis=1)
    df['Hitpath ID'] = df['Hitpath Offer ID']
    df['Submission Date'] = pd.Timestamp(date.today())
    df['Due Date '] = business_days_from_date(pd.Timestamp(date.today()), 3)
    df['Request Type (Request)'] = 'Content'
    df['Offer Type'] = 'Existing - More Content'
    df['Compliance & Additional Notes (Request)'] = df['Hitpath Offer ID'].apply(lambda x: 'Offer Wall' if x in ow_hitpath_list else 'N/A')
    df['Requested By'] = 'Script'
    df['Channel'] = 'SC'
    df['Platform'] = 'SS'
    df['Hitpath ID'] = df['Hitpath Offer ID']
    df['High Priority'] = False
    # Need to make names match what is in Creative Submission sheet for contact email purposes
    df['RX Rep'] = df['RX Rep'].apply(lambda x: find_closest_RX_Rep(x))

    # # Ensure that index starts at 0 and increments by 1
    # df = df.reset_index(drop=True)

    # starting_id = get_starting_ID_number()
    # df['Request ID'] = df.apply(lambda row: f'RQID.{starting_id+int(row.name)}.SMS', axis=1)
    

    df[['# of Pieces Requested Per Shortcode']] = df[['# of Pieces Requested Per Shortcode']].astype(int)
    df['# of Pieces Requested Per Shortcode'] = df['# of Pieces Requested Per Shortcode'].apply(limit_requests_per_shortcode)


    # For columns that do not yet exist, but are part of the SMS Creative Submission smartsheet, fill them with a blank string
    missing_columns = [col for col in ct_columns if col not in df.columns]
    for col in missing_columns:
        df[col] = ''

    df = df[ct_columns]

    # Calculate number of shortcodes per hitpath
    df['# of Shortcodes'] = df['Shortcode Abbreviations'].apply(lambda x: len(x.split('\n')))
    # Calculate total requests per hitpath 
    df['Total Requests per Hitpath'] = df['# of Pieces Requested Per Shortcode'] * df['# of Shortcodes']
    
    # Sort by highest amount of requests first, so that highest requests can be reduced, if necessary
    df = df.sort_values(by='Total Requests per Hitpath', ascending=False)

    return df

In [29]:
def calculate_request_diff(df, request_limit):
    
    df['# of Shortcodes'] = df['Shortcode Abbreviations'].apply(lambda x: len(x.split('\n')))
    df['Total Requests per Hitpath'] = df['# of Pieces Requested Per Shortcode'] * df['# of Shortcodes']
    total_requests = df['Total Requests per Hitpath'].sum()

    request_diff = total_requests - request_limit

    return request_diff

In [30]:
def spread_out_dates(df):
    total_requests = df['Total Requests per Hitpath'].sum()
    first_third = total_requests * (1/3)
    second_third = total_requests * (2/3)
    original_due_date = df.iloc[0]['Due Date ']

    df['Cumulative Request Total'] = df['Total Requests per Hitpath'].cumsum()

    if len(df) > 1:
        if len(df) == 2:
            df.loc[1, 'Due Date '] = business_days_from_date(df.loc[0, 'Due Date '], 1)
        else:
            second_third_df = df[(df['Cumulative Request Total'] > first_third) & (df['Cumulative Request Total'] <= second_third)]
            last_third_df = df[df['Cumulative Request Total'] > second_third]

            for ind, row in second_third_df.iterrows():
                df.loc[ind, 'Due Date '] = business_days_from_date(original_due_date, 1)
            for ind, row in last_third_df.iterrows():
                df.loc[ind, 'Due Date '] = business_days_from_date(original_due_date, 2)
        
    df = df.drop('Cumulative Request Total', axis=1)

    return df

In [31]:
def remove_not_live_ow_inventory(row):
    """Takes a row from a dataframe and returns True if the Shortcode is a live Shortcode for the given OW
    
    :param row: a DataFrame row with columns of 'Hitpath Offer ID' and 'Shortcode Name'
    :return: True or False
    """
    if row['Hitpath Offer ID'] in live_ow_shortcodes_dict.keys():
        key = row['Hitpath Offer ID']
        value = row['Shortcode Name']
        if value in live_ow_shortcodes_dict.get(key, []):
            return True
    else:
        return True
    
    return False

In [32]:
def make_pieces_request_df(request_limit=50):
    """
    Finds any Hitpath IDs that need more content and makes a dataframe of all content requests that need
    to be made, formatted to be written into the SMS Creative Submission smartsheet
    
    :param request_limit: an int of the maximum number of total content pieces to be requested (sum of pieces requested * # of shortcodes).
                         Default is 50.
    :return a dataframe that is ready to be written into the SMS Creative Submission smartsheet
    """
    submission_columns = ['Hitpath ID', 'Submission Date', 'Due Date ', 'Request Type (Request)',
                    'Creative Approval Status', 'Offer Type', 'Hitpath Offer ID', 'Offer Name',
                    'Vertical', 'Landing Page / Redirect Link', 'Compliance & Additional Notes (Request)',
                    '# of Pieces Requested Per Shortcode', '# of Pieces Requested Per Toll Free', 'Shortcode Abbreviations', 'Requested By', 'RX Rep',
                    'External AM Email', 'Channel', 'Platform']  
    # Find drops per month of each offer in each shortcode_DP.SV
    dpm = lexi_90.copy()
    dpm = dpm.drop_duplicates(subset=['shortcode_DP.SV', 'Date', 'Hitpath Offer ID'])
    dpm = dpm.groupby(['shortcode_DP.SV', 'Hitpath Offer ID']).agg(drops_per_month=('Hitpath Offer ID', 'count'))
    dpm['drops_per_month'] = dpm['drops_per_month'].apply(lambda x: int(np.ceil(x/3)))
    dpm = dpm.reset_index()
    dpm = dpm.merge(lexi_90[['shortcode_DP.SV', 'Shortcode Name']].drop_duplicates(subset='shortcode_DP.SV'), how='left', on='shortcode_DP.SV')
    dpm = dpm.groupby(['Shortcode Name', 'Hitpath Offer ID'])[['drops_per_month']].max()
    dpm = dpm.reset_index()

    # Find how many content pieces there are currently for each offer
    num_current_pieces = cw.groupby(['Type', 'OfferIDs']).agg(num_current_pieces=('Reporting Content ID', 'count'))
    num_current_pieces = num_current_pieces.reset_index()
    num_current_pieces = num_current_pieces.rename(columns={'Type':'Shortcode Name', 'OfferIDs':'Hitpath Offer ID'})

    offer_inventory = dpm.merge(num_current_pieces, how='left', on=['Shortcode Name', 'Hitpath Offer ID'])
    # Ensure that every hitpath that has dropped before has at least 2 pieces of content
    offer_inventory['Required Num of Pieces'] = offer_inventory['drops_per_month'].apply(lambda x: max(x, 2))

    stats_by_creative, bad_offers_dict = make_stats_by_creative()

    # Include number of bad offers
    offer_inventory['Poor Quality Pieces'] = offer_inventory.apply(lambda x: get_bad_offer_count(x, bad_offers_dict), axis=1)

    offer_inventory['Num Pieces to Request'] = offer_inventory['Required Num of Pieces'] + offer_inventory['Poor Quality Pieces'] - offer_inventory['num_current_pieces']
    offer_inventory['Num Pieces to Request'] = offer_inventory['Num Pieces to Request'].apply(lambda x: np.where(x < 0, 0, x))

    offer_inventory = offer_inventory[offer_inventory['Shortcode Name'].isin(['DSS', 'FLC', 'HZB', 'MBC', 'SVT', 'UAA'])]

    # Only keep shortcodes that are live for the offer wall
    offer_inventory = offer_inventory[offer_inventory.apply(remove_not_live_ow_inventory, axis=1)]

    # A list of Hitpath IDs that have had content requested recently
    already_requested_content = find_already_requested_content()

    # This will look at a given hitpath ID. If any shortcode has sent the offer in the past 90 days, it will be listed even if it has enough content
    final_request_df = offer_inventory.groupby('Hitpath Offer ID').agg(Num_Pieces_to_Request=('Num Pieces to Request', 'max'), Shortcode_Names=('Shortcode Name', set))
    # Only consider hitpath ids that need more content
    final_request_df = final_request_df[final_request_df['Num_Pieces_to_Request'] > 0]
    final_request_df = final_request_df[~final_request_df.index.isin(already_requested_content)]
    final_request_df = final_request_df.reset_index()
    
    # Only consider offers that have sent as a P drop in the past 30 days
    final_request_df = final_request_df[final_request_df['Hitpath Offer ID'].isin(eligible_hitpaths)]
    
    # Find offer status
    final_request_df = final_request_df.merge(offers[['Hitpath Offer ID', 'Status']], how='left', on='Hitpath Offer ID')
    # Give OW's a status 
    final_request_df['Status'] = final_request_df.apply(lambda x: ow_status_dict.get(x['Hitpath Offer ID'], x['Status']), axis=1)
    # Only consider offers that are Live or Live: Budgeted Offers
    final_request_df = final_request_df[final_request_df['Status'].isin(['Live', 'Live: Budgeted Offer'])]

    # Ensure that OW's are requesting content only for Available Shortcodes
    final_request_df['Shortcode_Names'] = final_request_df.apply(lambda x: live_ow_shortcodes_dict.get(x['Hitpath Offer ID'], x['Shortcode_Names']), axis=1)

    # Get extra info necessary for submission sheet, and format properly
    submission_df = finalize_submission_df(final_request_df)
    submission_df = submission_df.reset_index(drop=True)

    # Determine priority based on offer performance to choose which requests to remove first
    stats_by_offer, priority_dict = make_stats_by_offer(submission_df['Hitpath Offer ID'])

    stats_by_offer = stats_by_offer.merge(submission_df[['Hitpath Offer ID', '# of Pieces Requested Per Shortcode', 'Shortcode Abbreviations', '# of Shortcodes', 'Total Requests per Hitpath']], how='left', on='Hitpath Offer ID')
    stats_by_offer = stats_by_offer.sort_values(by=['Priority Order', '# of Shortcodes'], ascending=[True, False])

    # Do not want to request content that is performing poorly and thus not sending much--must have sent approx. 2 drops to each shortcode before requesting content 
    offer_30D_drop_count = lexi_30[lexi_30['Hitpath Offer ID'].isin(stats_by_offer['Hitpath Offer ID'])].groupby(['Hitpath Offer ID']).agg(Num_Drops=('Date', 'count')).to_dict()
    offer_90D_drop_count = lexi_90[lexi_90['Hitpath Offer ID'].isin(stats_by_offer['Hitpath Offer ID'])].groupby(['Hitpath Offer ID']).agg(Num_Drops=('Date', 'count')).to_dict()
    submission_df['30D Drop Count'] = submission_df['Hitpath Offer ID'].map(offer_30D_drop_count['Num_Drops'])
    submission_df['90D Drop Count'] = submission_df['Hitpath Offer ID'].map(offer_90D_drop_count['Num_Drops'])
    # Must have sent enough drops that there could be 2 drops to each shortcode
    submission_df = submission_df[submission_df['30D Drop Count'] >= (2 * submission_df['# of Shortcodes'])]

    stats_by_offer = stats_by_offer[stats_by_offer['Hitpath Offer ID'].isin(submission_df['Hitpath Offer ID'])]
    stats_by_offer['90D Drop Count'] = stats_by_offer['Hitpath Offer ID'].map(offer_90D_drop_count['Num_Drops'])
    stats_by_offer = stats_by_offer.rename(columns={'Num_Drops':'30D Drop Count'})
    stats_by_offer['30:90 Ratio'] = stats_by_offer['30D Drop Count'] / stats_by_offer['90D Drop Count']
    stats_by_offer = stats_by_offer[stats_by_offer.columns[:5].tolist() + ['30D Drop Count', '90D Drop Count', '30:90 Ratio'] + stats_by_offer.columns[6:-2].tolist()]
    # Restart ranking from 1 
    stats_by_offer['Priority Order'] = stats_by_offer['Priority Order'].rank(ascending=True)

    # Order offers from least important to most important based on performance
    submission_df['Priority Order'] = submission_df['Hitpath Offer ID'].apply(lambda x: priority_dict[x])
    submission_df = submission_df.sort_values(by=['Priority Order', '# of Shortcodes'], ascending=[False, True])

    request_diff = calculate_request_diff(submission_df, request_limit)


    """ This method first removes 1 request from each row, then cycles back through until all rows are at 1 request per hitpath.
        It then removes rows."""
    # while submission_df['# of Pieces Requested Per Shortcode'].max() > 1:
    #     # Remove 1 hitpath request if there are at least 2 until request limit met
    #     for ind, row in submission_df.iterrows():
    #         if row['# of Pieces Requested Per Shortcode'] > 1:
    #             submission_df.loc[ind, '# of Pieces Requested Per Shortcode'] = row['# of Pieces Requested Per Shortcode'] - 1
    #         else:
    #             continue
    #         request_diff = calculate_request_diff(submission_df, request_limit)
    #         if request_diff <= 0:
    #             break
    #     if request_diff <= 0:
    #             break
    
    # # If request limit still is not met, remove low priority hitpath requests until request limit met
    # if request_diff > 0:
    #     for ind, row in submission_df.iterrows():
    #         submission_df = submission_df.drop(ind)
    #         request_diff = calculate_request_diff(submission_df, request_limit)
    #         if request_diff <= 0:
    #             break


    # """ This method removes entire rows at a time (hitpath requests) until request limit is met"""
    # if request_diff > 0:
    #     for ind, row in submission_df.iterrows():
    #         submission_df = submission_df.drop(ind)
    #         request_diff = calculate_request_diff(submission_df, request_limit)
    #         if request_diff <= 0:
    #             break
    
    # # Place top priority offers first
    submission_df = submission_df.sort_values(by=['Priority Order', '# of Shortcodes'], ascending=[True, False])
    submission_df = submission_df.reset_index(drop=True)

    total_requests = 0
    final_submission_df = pd.DataFrame(columns=submission_df.columns)

    for index, row in submission_df.iterrows():
        if total_requests + row['Total Requests per Hitpath'] <= request_limit:
            final_submission_df = pd.concat([final_submission_df, pd.DataFrame([row], columns=submission_df.columns)], ignore_index=True)
            total_requests += row['Total Requests per Hitpath']

    # """This method adds request rows until it gets to (or close to) the request limit"""
    # final_submission_df = pd.DataFrame(columns=submission_df.columns)
    # for ind, row in submission_df.iterrows():
    #     print(ind)
    #     request_diff = calculate_request_diff(final_submission_df, request_limit)
    #     print(request_diff)
    #     if request_diff < 0:
    #         final_submission_df = pd.concat([final_submission_df, row], ignore_index=True)
    #     else:
    #         continue

    final_submission_df = spread_out_dates(final_submission_df)

    # Get Dates into proper format for smartsheet submission (must be 'YYYY-MM-DD')
    for col in ['Submission Date', 'Due Date ']:
        final_submission_df[col] = final_submission_df[col].apply(lambda x: x.strftime("%Y-%m-%d"))

    # submission_df = submission_df.drop(['# of Shortcodes', 'Total Requests per Hitpath', 'Priority Order'], axis=1)

    final_submission_df['# of Pieces Requested Per Toll Free'] = 0

    # Only keep necessary columns
    final_submission_df = final_submission_df[submission_columns]

    # Cannot have NaNs when writing to smartsheet
    final_submission_df = final_submission_df.fillna('')

    offer_inventory = offer_inventory.rename(columns={'drops_per_month':'Max Drops Per Month', 'num_current_pieces':'Num Content Pieces in La Nina'})
    offer_inventory = offer_inventory.fillna(0)
    offer_inventory[offer_inventory.columns[2:]] = offer_inventory[offer_inventory.columns[2:]].astype(int)
    offer_inventory = offer_inventory.sort_values(by=['Hitpath Offer ID', 'Shortcode Name'])

    # Need to have Hitpath Offer ID column as an int so that Status can be pulled properly
    final_submission_df['Hitpath Offer ID'] = final_submission_df['Hitpath Offer ID'].apply(lambda x: int(x) if x.isdigit() else x)

    return final_submission_df, stats_by_offer, stats_by_creative, offer_inventory

In [33]:
def convert_number_to_letter(num_sheet_cols):
    """
    Takes the 1-indexed number of a column in a google sheet and returns the letter version. For example,
    if num_sheet_cols=1, it would return 'A'. If it was 27, it would return 'AA'. If it was 53, it would return 'BA'.

    :param num_sheet_cols: an int of the column in the google sheet
    :return: a string of the corresponding letter(s) of the column
    """
    # One letter cases (A-Z)
    if ((num_sheet_cols // 26) == 0) | (num_sheet_cols == 26):
        letter = chr(64 + num_sheet_cols)

    # Multiple letter cases (i.e. AA)
    else:
        # Z special case
        if num_sheet_cols % 26 == 0:
            first_letter = chr(64 + (num_sheet_cols // 26) - 1)
            second_letter = chr(64 + num_sheet_cols - (((num_sheet_cols // 26) - 1) * 26))
            letter = (first_letter + second_letter)
        # All other letters
        else:
            first_letter = chr(64 + (num_sheet_cols // 26))
            second_letter = chr(64 + num_sheet_cols - ((num_sheet_cols // 26) * 26))
            letter = (first_letter + second_letter)

    return letter

In [34]:
def get_previous_script_requests():
    last_script_request_date = ct[ct['Requested By']=='Script']['Submission Date'].max()
    prev_requests = ct[(ct['Requested By']=='Script') & (ct['Submission Date']==last_script_request_date)]

    return prev_requests

In [35]:
def get_rx_rep_offer_counts(submission_df):
    rep_dict = submission_df.groupby('RX Rep')['Hitpath Offer ID'].count().to_dict()
    if '' in rep_dict.keys():
        rep_dict['Offer Wall'] = rep_dict['']
        del rep_dict['']
    
    # Sort alphabetically based on first name
    rep_dict = dict(sorted(rep_dict.items()))
    
    return rep_dict

# Function for TF

In [36]:
def find_already_requested_tf_content():
    """
    :return: a list of Hitpath Offer IDs that have had TF content requested within the past 3 weeks in the 
            SMS Creative Submission smartsheet.
    """
    already_requested_content = ct.copy()
    already_requested_content = already_requested_content[['Submission Date', 'Date Added to La Nina (Akshad)', '# of Pieces Requested Per Toll Free', 'Hitpath Offer ID', 'Request Type (Request)','Toll Free Abbreviations']]
    already_requested_content = already_requested_content[already_requested_content['Request Type (Request)'].str.contains('Content', na=False) ]
    # Any offer without a date in this column has not been added to La Nina, and should not be requested again
    already_requested_content = already_requested_content[((pd.isna(already_requested_content['Date Added to La Nina (Akshad)']))&(already_requested_content['Submission Date']>=pd.Timestamp(date.today()-timedelta(30)))) | 
                                                          (already_requested_content['Date Added to La Nina (Akshad)']>=pd.Timestamp(date.today() - timedelta(21)))]
    already_requested_content = already_requested_content[ already_requested_content['# of Pieces Requested Per Toll Free']>0]                                                      

    return already_requested_content['Hitpath Offer ID'].unique().tolist()

In [37]:
def finalize_submission_tf_df(df,request_tf):
    """
    Properly formats the dataframe containing info on # of Pieces needed to be requested so that it can be pasted into
    the SMS Creative Submission smartsheet. Also, gathers any additional information needed.

    :param df: a dataframe of the # of pieces needed to be requested for each Shortcode/Hitpath ID combo
    :return: a dataframe that is ready to be pasted into the SMS Creative Submission smartsheet
    """
    ct_columns = ['Hitpath ID', 'Request ID', 'High Priority', 'Submission Date',
    'Due Date ', 'Request Sent to Copywriter (Kellie)', 'Unapproved Content Document Link (Copywriter)',
    'Unapproved Jump Page Content (Copywriter)', 'Image ID - Jump Page (Copywriter)', 'Request Type (Request)',
    'Content Sent to Compliance (Kellie)', 'Content Approved by Compliance (Kellie)', 'Date - Jump Page Created (Akshad)',
    'Jump Page Link (Akshad)', 'Creative Approval Status', 'Date - Content sent for approval to network (K.S.)',
    'Date Got Approval from Advertiser (K.S.)', 'Approved Jump Page Link (Kellie)', 'External - Approved Content Document (Kellie)',
    'Ready for Akshad (Kellie Checks; Alert Sent)', 'Date Added to La Nina (Akshad)', 'Offer Type', 'Operational Status (Connected to SMS-OMA)',
    'Hitpath Offer ID', 'Offer Name', 'Vertical', 'Landing Page / Redirect Link', 'Compliance & Additional Notes (Request)',
    '# of Pieces Requested Per Shortcode', '# of Pieces Requested Per Toll Free', 'Shortcode Abbreviations', 'Toll Free Abbreviations', 'Requested By', 
    'RX Rep', 'External AM Email', 'Channel', 'Platform']

    # date_columns = ['Submission Date', 'Due Date ', 'Request Sent to Copywriter (Kellie)',
    # 'Content Sent to Compliance (Kellie)', 'Content Approved by Compliance (Kellie)', 'Date - Jump Page Created (Akshad)',
    # 'Date - Content sent for approval to network (K.S.)',
    # 'Date Got Approval from Advertiser (K.S.)', 'Date Added to La Nina (Akshad)']
    
    # submission_columns = ['Hitpath ID', 'Submission Date', 'Due Date ', 'Request Type (Request)',
    #                   'Creative Approval Status', 'Offer Type', 'Hitpath Offer ID', 'Offer Name',
    #                   'Vertical', 'Landing Page / Redirect Link', 'Compliance & Additional Notes (Request)',
    #                   '# of Pieces Requested Per Shortcode', 'Shortcode', 'Requested By', 'RX Rep',
    #                   'External AM Email', 'Channel', 'Platform']    

    df = df.merge(offers[['Hitpath Offer ID', 'Offer Name', 'Custom Creative Allowed', 'Vertical', 'Redirect Link', 'RX Rep', 'Advertiser Rep', 'Advertiser']], how='left', on='Hitpath Offer ID')
    # Merge OW info
    df['Landing Page / Redirect Link'] = df['Redirect Link']
    df = df.rename(columns={'Custom Creative Allowed':'Creative Approval Status', 'Pieces to Request':'# of Pieces Requested Per Toll Free'})
    
    # df['Creative Approval Status'] = df['Creative Approval Status'].apply(lambda x: fix_approval_status(x))
    df['Advertiser Rep'] = df['Advertiser Rep'].fillna('')
    df['Toll Free Abbreviations'] = '\n'.join(request_tf)
    df['External AM Email'] = df.apply(lambda x: get_advertiser_email_address(x), axis=1)
    df['Hitpath ID'] = df['Hitpath Offer ID']
    df['Submission Date'] = pd.Timestamp(date.today())
    df['Due Date '] = business_days_from_date(pd.Timestamp(date.today()), 3)
    df['Request Type (Request)'] = 'Content'
    df['Offer Type'] = 'Existing - More Content'
    df['Compliance & Additional Notes (Request)'] = 'Extra Content for direct traffic in Toll Free'
    df['Requested By'] = 'Script'
    df['Channel'] = 'TF'
    df['Platform'] = 'SS'
    df['Hitpath ID'] = df['Hitpath Offer ID']
    df['High Priority'] = False
    # Need to make names match what is in Creative Submission sheet for contact email purposes
    df['RX Rep'] = df['RX Rep'].apply(lambda x: find_closest_RX_Rep(x))

    # # Ensure that index starts at 0 and increments by 1
    # df = df.reset_index(drop=True)

    # starting_id = get_starting_ID_number()
    # df['Request ID'] = df.apply(lambda row: f'RQID.{starting_id+int(row.name)}.SMS', axis=1)
    

    df[['# of Pieces Requested Per Toll Free']] = df[['# of Pieces Requested Per Toll Free']].astype(int)

    # For columns that do not yet exist, but are part of the SMS Creative Submission smartsheet, fill them with a blank string
    missing_columns = [col for col in ct_columns if col not in df.columns]
    for col in missing_columns:
        df[col] = ''

    df = df[ct_columns]

    # Calculate number of shortcodes per hitpath
    df['# of Shortcodes'] = df['Toll Free Abbreviations'].apply(lambda x: len(x.split('\n')))
    # Calculate total requests per hitpath 
    df['Total Requests per Hitpath'] = df['# of Pieces Requested Per Toll Free'] * df['# of Shortcodes']
    
    # Sort by highest amount of requests first, so that highest requests can be reduced, if necessary
    df = df.sort_values(by='Total Requests per Hitpath', ascending=False)

    return df


In [38]:
def calculate_request_tf_diff(df, request_limit):
    
    df['# of Shortcodes'] = df['Toll Free Abbreviations'].apply(lambda x: len(x.split('\n')))
    df['Total Requests per Hitpath'] = df['# of Pieces Requested Per Toll Free'] * df['# of Shortcodes']
    total_requests = df['Total Requests per Hitpath'].sum()

    request_diff = total_requests - request_limit

    return request_diff

In [39]:
def make_pieces_request_tf_df(submission_df,request_tf, request_limit=50):

###########################
    """
    Finds any Hitpath IDs that need more content and makes a dataframe of all content requests that need
    to be made, formatted to be written into the SMS Creative Submission smartsheet
    
    :param request_limit: an int of the maximum number of total content pieces to be requested (sum of pieces requested * # of shortcodes).
                         Default is 50.
    :return a dataframe that is ready to be written into the SMS Creative Submission smartsheet
    """
    submission_columns = ['Hitpath ID', 'Submission Date', 'Due Date ', 'Request Type (Request)',
                    'Creative Approval Status', 'Offer Type', 'Hitpath Offer ID', 'Offer Name',
                    'Vertical', 'Landing Page / Redirect Link', 'Compliance & Additional Notes (Request)',
                    '# of Pieces Requested Per Shortcode', '# of Pieces Requested Per Toll Free', 'Shortcode Abbreviations', 'Toll Free Abbreviations','Requested By', 'RX Rep',
                    'External AM Email', 'Channel', 'Platform']   

    # Get extra info necessary for submission sheet, and format properly
    
    submission_df = submission_df.reset_index(drop=True)

    # Determine priority based on offer performance to choose which requests to remove first
    stats_by_offer, priority_dict = make_stats_by_offer(submission_df['Hitpath Offer ID'])

    stats_by_offer = stats_by_offer.merge(submission_df[['Hitpath Offer ID', '# of Pieces Requested Per Toll Free', 'Toll Free Abbreviations', '# of Shortcodes', 'Total Requests per Hitpath']], how='left', on='Hitpath Offer ID')
    stats_by_offer = stats_by_offer.sort_values(by=['Priority Order', '# of Shortcodes'], ascending=[True, False])

    # Do not want to request content that is performing poorly and thus not sending much--must have sent approx. 2 drops to each shortcode before requesting content 
    offer_30D_drop_count = lexi_30[lexi_30['Hitpath Offer ID'].isin(stats_by_offer['Hitpath Offer ID'])].groupby(['Hitpath Offer ID']).agg(Num_Drops=('Date', 'count')).to_dict()
    offer_90D_drop_count = lexi_90[lexi_90['Hitpath Offer ID'].isin(stats_by_offer['Hitpath Offer ID'])].groupby(['Hitpath Offer ID']).agg(Num_Drops=('Date', 'count')).to_dict()
    submission_df['30D Drop Count'] = submission_df['Hitpath Offer ID'].map(offer_30D_drop_count['Num_Drops'])
    submission_df['90D Drop Count'] = submission_df['Hitpath Offer ID'].map(offer_90D_drop_count['Num_Drops'])
    # Must have sent enough drops that there could be 2 drops to each shortcode
    submission_df = submission_df[submission_df['30D Drop Count'] >= (2 * submission_df['# of Shortcodes'])]

    stats_by_offer = stats_by_offer[stats_by_offer['Hitpath Offer ID'].isin(submission_df['Hitpath Offer ID'])]
    stats_by_offer['90D Drop Count'] = stats_by_offer['Hitpath Offer ID'].map(offer_90D_drop_count['Num_Drops'])
    stats_by_offer = stats_by_offer.rename(columns={'Num_Drops':'30D Drop Count'})
    stats_by_offer['30:90 Ratio'] = stats_by_offer['30D Drop Count'] / stats_by_offer['90D Drop Count']
    stats_by_offer = stats_by_offer[stats_by_offer.columns[:5].tolist() + ['30D Drop Count', '90D Drop Count', '30:90 Ratio'] + stats_by_offer.columns[6:-2].tolist()]
    # Restart ranking from 1 
    stats_by_offer['Priority Order'] = stats_by_offer['Priority Order'].rank(ascending=True)

    # Order offers from least important to most important based on performance
    submission_df['Priority Order'] = submission_df['Hitpath Offer ID'].apply(lambda x: priority_dict[x])
    submission_df = submission_df.sort_values(by=['Priority Order', '# of Shortcodes'], ascending=[False, True])

    request_diff = calculate_request_tf_diff(submission_df, request_limit)


    """ This method first removes 1 request from each row, then cycles back through until all rows are at 1 request per hitpath.
        It then removes rows."""
    # while submission_df['# of Pieces Requested Per Shortcode'].max() > 1:
    #     # Remove 1 hitpath request if there are at least 2 until request limit met
    #     for ind, row in submission_df.iterrows():
    #         if row['# of Pieces Requested Per Shortcode'] > 1:
    #             submission_df.loc[ind, '# of Pieces Requested Per Shortcode'] = row['# of Pieces Requested Per Shortcode'] - 1
    #         else:
    #             continue
    #         request_diff = calculate_request_diff(submission_df, request_limit)
    #         if request_diff <= 0:
    #             break
    #     if request_diff <= 0:
    #             break
    
    # # If request limit still is not met, remove low priority hitpath requests until request limit met
    # if request_diff > 0:
    #     for ind, row in submission_df.iterrows():
    #         submission_df = submission_df.drop(ind)
    #         request_diff = calculate_request_diff(submission_df, request_limit)
    #         if request_diff <= 0:
    #             break


    # """ This method removes entire rows at a time (hitpath requests) until request limit is met"""
    # if request_diff > 0:
    #     for ind, row in submission_df.iterrows():
    #         submission_df = submission_df.drop(ind)
    #         request_diff = calculate_request_diff(submission_df, request_limit)
    #         if request_diff <= 0:
    #             break
    
    # # Place top priority offers first
    submission_df = submission_df.sort_values(by=['Priority Order', '# of Shortcodes'], ascending=[True, False])
    submission_df = submission_df.reset_index(drop=True)

    total_requests = 0
    
    final_submission_df = pd.DataFrame(columns=submission_df.columns)

    for index, row in submission_df.iterrows():
        if total_requests + row['Total Requests per Hitpath'] <= request_limit:
            final_submission_df = pd.concat([final_submission_df, pd.DataFrame([row], columns=submission_df.columns)], ignore_index=True)
            total_requests += row['Total Requests per Hitpath']
    print(final_submission_df)
    # """This method adds request rows until it gets to (or close to) the request limit"""
    # final_submission_df = pd.DataFrame(columns=submission_df.columns)
    # for ind, row in submission_df.iterrows():
    #     print(ind)
    #     request_diff = calculate_request_diff(final_submission_df, request_limit)
    #     print(request_diff)
    #     if request_diff < 0:
    #         final_submission_df = pd.concat([final_submission_df, row], ignore_index=True)
    #     else:
    #         continue

    final_submission_df = spread_out_dates(final_submission_df)

    # Get Dates into proper format for smartsheet submission (must be 'YYYY-MM-DD')
    for col in ['Submission Date', 'Due Date ']:
        final_submission_df[col] = final_submission_df[col].apply(lambda x: x.strftime("%Y-%m-%d"))

    # submission_df = submission_df.drop(['# of Shortcodes', 'Total Requests per Hitpath', 'Priority Order'], axis=1)

    final_submission_df['# of Pieces Requested Per Shortcode'] = 0

    # Only keep necessary columns
    final_submission_df = final_submission_df[submission_columns]

    # Cannot have NaNs when writing to smartsheet
    final_submission_df = final_submission_df.fillna('')


    # Need to have Hitpath Offer ID column as an int so that Status can be pulled properly
    final_submission_df['Hitpath Offer ID'] = final_submission_df['Hitpath Offer ID'].apply(lambda x: int(x) if x.isdigit() else x)

    return final_submission_df

# Function for Full combined request 

In [40]:
def finalize_submission_all_df(df):
    """
    Properly formats the dataframe containing info on # of Pieces needed to be requested so that it can be pasted into
    the SMS Creative Submission smartsheet. Also, gathers any additional information needed.

    :param df: a dataframe of the # of pieces needed to be requested for each Shortcode/Hitpath ID combo
    :return: a dataframe that is ready to be pasted into the SMS Creative Submission smartsheet
    """
    ct_columns = ['Hitpath ID', 'Request ID', 'High Priority', 'Submission Date',
    'Due Date ', 'Request Sent to Copywriter (Kellie)', 'Unapproved Content Document Link (Copywriter)',
    'Unapproved Jump Page Content (Copywriter)', 'Image ID - Jump Page (Copywriter)', 'Request Type (Request)',
    'Content Sent to Compliance (Kellie)', 'Content Approved by Compliance (Kellie)', 'Date - Jump Page Created (Akshad)',
    'Jump Page Link (Akshad)', 'Creative Approval Status', 'Date - Content sent for approval to network (K.S.)',
    'Date Got Approval from Advertiser (K.S.)', 'Approved Jump Page Link (Kellie)', 'External - Approved Content Document (Kellie)','Seasonal Date Restriction',
    'Ready for Akshad (Kellie Checks; Alert Sent)', 'Date Added to La Nina (Akshad)', 'Offer Type', 'Operational Status (Connected to SMS-OMA)',
    'Hitpath Offer ID', 'Offer Name', 'Vertical', 'Landing Page / Redirect Link', 'Compliance & Additional Notes (Request)',
    '# of Pieces Requested Per Shortcode', '# of Pieces Requested Per Toll Free', 'Shortcode Abbreviations', 'Toll Free Abbreviations', 'Requested By', 
    'RX Rep', 'External AM Email', 'Channel', 'Platform','Total Requests per Hitpath','Total Requests per Toll Free','# of Pieces Requested Per Jump Page']

    # date_columns = ['Submission Date', 'Due Date ', 'Request Sent to Copywriter (Kellie)',
    # 'Content Sent to Compliance (Kellie)', 'Content Approved by Compliance (Kellie)', 'Date - Jump Page Created (Akshad)',
    # 'Date - Content sent for approval to network (K.S.)',
    # 'Date Got Approval from Advertiser (K.S.)', 'Date Added to La Nina (Akshad)']
    
    # submission_columns = ['Hitpath ID', 'Submission Date', 'Due Date ', 'Request Type (Request)',
    #                   'Creative Approval Status', 'Offer Type', 'Hitpath Offer ID', 'Offer Name',
    #                   'Vertical', 'Landing Page / Redirect Link', 'Compliance & Additional Notes (Request)',
    #                   '# of Pieces Requested Per Shortcode', 'Shortcode', 'Requested By', 'RX Rep',
    #                   'External AM Email', 'Channel', 'Platform']    

    df = df.merge(offers[['Hitpath Offer ID', 'Offer Name', 'Custom Creative Allowed', 'Vertical', 'Redirect Link', 'RX Rep', 'Advertiser Rep', 'Advertiser']], how='left', on='Hitpath Offer ID')
    # Merge OW info
    df['Landing Page / Redirect Link'] = df['Redirect Link']
    df = df.rename(columns={'Custom Creative Allowed':'Creative Approval Status'})
    
    # df['Creative Approval Status'] = df['Creative Approval Status'].apply(lambda x: fix_approval_status(x))
    df['Advertiser Rep'] = df['Advertiser Rep'].fillna('')
    df['External AM Email'] = df.apply(lambda x: get_advertiser_email_address(x), axis=1)
    df['Hitpath ID'] = df['Hitpath Offer ID']
    df['Submission Date'] = pd.Timestamp(date.today())
    df['Due Date '] = business_days_from_date(pd.Timestamp(date.today()), 3)
    df['Request Type (Request)'] = df.apply(
                lambda x: "Content" if x['# of Pieces Requested Per Jump Page'] == 0 
                else "Jump Page" if x['# of Pieces Requested Per Toll Free'] + x['# of Pieces Requested Per Shortcode'] == 0 
                else "Content & Jump Page", 
                axis=1
                )
    df['Offer Type'] = 'Existing - More Content'
    df['Compliance & Additional Notes (Request)'] = ''
    df['Requested By'] = 'Script'
    df['Channel List'] = df.apply( lambda x: ['SC','TF'] if x['# of Pieces Requested Per Toll Free'] > 0 and x['# of Pieces Requested Per Shortcode']+ x['# of Pieces Requested Per Jump Page'] > 0 
                                  else ['SC'] if  x['# of Pieces Requested Per Toll Free'] == 0
                                  else ['TF'], axis= 1)
    df['Channel'] = df['Channel List'].apply(lambda x: '\n'.join(x))
    df['Platform'] = 'SS'
    df['Hitpath ID'] = df['Hitpath Offer ID']
    df['High Priority'] = False
    # Need to make names match what is in Creative Submission sheet for contact email purposes
    df['RX Rep'] = df['RX Rep'].apply(lambda x: find_closest_RX_Rep(x))

    # # Ensure that index starts at 0 and increments by 1
    # df = df.reset_index(drop=True)

    # starting_id = get_starting_ID_number()
    # df['Request ID'] = df.apply(lambda row: f'RQID.{starting_id+int(row.name)}.SMS', axis=1)
    

    df[['# of Pieces Requested Per Toll Free']] = df[['# of Pieces Requested Per Toll Free']].astype(int)
    df[['# of Pieces Requested Per Shortcode']] = df[['# of Pieces Requested Per Shortcode']].astype(int)
    df['# of Pieces Requested Per Shortcode'] = df['# of Pieces Requested Per Shortcode'].apply(limit_requests_per_shortcode)

    # For columns that do not yet exist, but are part of the SMS Creative Submission smartsheet, fill them with a blank string
    missing_columns = [col for col in ct_columns if col not in df.columns]
    for col in missing_columns:
        df[col] = ''

    df = df[ct_columns]

    return df


In [41]:
def make_pieces_request_all_df(submission_df,request_limit = 50 ):     
    

    """
    Finds any Hitpath IDs that need more content and makes a dataframe of all content requests that need
    to be made, formatted to be written into the SMS Creative Submission smartsheet
    
    :param request_limit: an int of the maximum number of total content pieces to be requested (sum of pieces requested * # of shortcodes).
                         Default is 50.
    :return a dataframe that is ready to be written into the SMS Creative Submission smartsheet
    """
    submission_columns = ['Hitpath ID', 'Submission Date', 'Due Date ', 'Request Type (Request)',
                    'Creative Approval Status', 'Offer Type', 'Hitpath Offer ID', 'Offer Name',
                    'Vertical', 'Landing Page / Redirect Link', 'Compliance & Additional Notes (Request)',
                    '# of Pieces Requested Per Shortcode', '# of Pieces Requested Per Toll Free', 'Shortcode Abbreviations', 'Toll Free Abbreviations','Requested By', 'RX Rep',
                    'External AM Email', 'Channel', 'Platform']   

    # Get extra info necessary for submission sheet, and format properly
    
    submission_df = submission_df.reset_index(drop=True)

    # Determine priority based on offer performance to choose which requests to remove first
    stats_by_offer, priority_dict = make_stats_by_offer(submission_df['Hitpath Offer ID'])

    stats_by_offer = stats_by_offer.merge(submission_df[['Hitpath Offer ID', '# of Pieces Requested Per Toll Free', 'Toll Free Abbreviations', 'Shortcode Abbreviations', '# of Pieces Requested Per Shortcode','# of Pieces Requested Per Jump Page']], how='left', on='Hitpath Offer ID')
    stats_by_offer = stats_by_offer.sort_values(by=['Priority Order'], ascending=[True])

    # Do not want to request content that is performing poorly and thus not sending much--must have sent approx. 2 drops to each shortcode before requesting content 
    offer_30D_drop_count = lexi_30[lexi_30['Hitpath Offer ID'].isin(stats_by_offer['Hitpath Offer ID'])].groupby(['Hitpath Offer ID']).agg(Num_Drops=('Date', 'count')).to_dict()
    offer_90D_drop_count = lexi_90[lexi_90['Hitpath Offer ID'].isin(stats_by_offer['Hitpath Offer ID'])].groupby(['Hitpath Offer ID']).agg(Num_Drops=('Date', 'count')).to_dict()
    submission_df['30D Drop Count'] = submission_df['Hitpath Offer ID'].map(offer_30D_drop_count['Num_Drops'])
    submission_df['90D Drop Count'] = submission_df['Hitpath Offer ID'].map(offer_90D_drop_count['Num_Drops'])
    # Must have sent enough drops that there could be 2 drops to each shortcode
    submission_df = submission_df[submission_df['30D Drop Count'] >= 10]

    stats_by_offer = stats_by_offer[stats_by_offer['Hitpath Offer ID'].isin(submission_df['Hitpath Offer ID'])]
    stats_by_offer['90D Drop Count'] = stats_by_offer['Hitpath Offer ID'].map(offer_90D_drop_count['Num_Drops'])
    stats_by_offer = stats_by_offer.rename(columns={'Num_Drops':'30D Drop Count'})
    stats_by_offer['30:90 Ratio'] = stats_by_offer['30D Drop Count'] / stats_by_offer['90D Drop Count']
    stats_by_offer = stats_by_offer[stats_by_offer.columns[:5].tolist() + ['30D Drop Count', '90D Drop Count', '30:90 Ratio'] + stats_by_offer.columns[6:-2].tolist()]
    # Restart ranking from 1 
    stats_by_offer['Priority Order'] = stats_by_offer['Priority Order'].rank(ascending=True)

    # Order offers from least important to most important based on performance
    submission_df['Priority Order'] = submission_df['Hitpath Offer ID'].apply(lambda x: priority_dict[x])
    submission_df = submission_df.sort_values(by=['Priority Order'], ascending=[False])
    
    
    submission_df['Total Requests per Toll Free'] = submission_df['Total Requests per Toll Free'].fillna(0)
    submission_df['Total Requests per Hitpath'] = submission_df['Total Requests per Hitpath'].fillna(0)
    submission_df['Total Request'] = submission_df['Total Requests per Toll Free'] + submission_df['Total Requests per Hitpath']
    submission_df.loc[submission_df['Total Requests per Hitpath']!=0, 'Total Request'] = submission_df['Total Requests per Hitpath']
    submission_df.loc[submission_df['Total Request']==0, 'Total Request'] = 1
    total_requests = submission_df['Total Requests per Toll Free'].sum()+ submission_df['Total Requests per Hitpath'].sum()
    request_diff = total_requests - request_limit

    """ This method first removes 1 request from each row, then cycles back through until all rows are at 1 request per hitpath.
        It then removes rows."""
    # while submission_df['# of Pieces Requested Per Shortcode'].max() > 1:
    #     # Remove 1 hitpath request if there are at least 2 until request limit met
    #     for ind, row in submission_df.iterrows():
    #         if row['# of Pieces Requested Per Shortcode'] > 1:
    #             submission_df.loc[ind, '# of Pieces Requested Per Shortcode'] = row['# of Pieces Requested Per Shortcode'] - 1
    #         else:
    #             continue
    #         request_diff = calculate_request_diff(submission_df, request_limit)
    #         if request_diff <= 0:
    #             break
    #     if request_diff <= 0:
    #             break
    
    # # If request limit still is not met, remove low priority hitpath requests until request limit met
    # if request_diff > 0:
    #     for ind, row in submission_df.iterrows():
    #         submission_df = submission_df.drop(ind)
    #         request_diff = calculate_request_diff(submission_df, request_limit)
    #         if request_diff <= 0:
    #             break


    # """ This method removes entire rows at a time (hitpath requests) until request limit is met"""
    # if request_diff > 0:
    #     for ind, row in submission_df.iterrows():
    #         submission_df = submission_df.drop(ind)
    #         request_diff = calculate_request_diff(submission_df, request_limit)
    #         if request_diff <= 0:
    #             break
    
    # # Place top priority offers first
    submission_df = submission_df.sort_values(by=['Priority Order'], ascending=[True])
    submission_df = submission_df.reset_index(drop=True)

    total_requests = 0
    
    final_submission_df = pd.DataFrame(columns=submission_df.columns)

    for index, row in submission_df.iterrows():
        if total_requests + row['Total Request'] <= request_limit:
            final_submission_df = pd.concat([final_submission_df, pd.DataFrame([row], columns=submission_df.columns)], ignore_index=True)
            total_requests += row['Total Request']
    print(final_submission_df)
    # """This method adds request rows until it gets to (or close to) the request limit"""
    # final_submission_df = pd.DataFrame(columns=submission_df.columns)
    # for ind, row in submission_df.iterrows():
    #     print(ind)
    #     request_diff = calculate_request_diff(final_submission_df, request_limit)
    #     print(request_diff)
    #     if request_diff < 0:
    #         final_submission_df = pd.concat([final_submission_df, row], ignore_index=True)
    #     else:
    #         continue

    final_submission_df = spread_out_dates(final_submission_df)

    # Get Dates into proper format for smartsheet submission (must be 'YYYY-MM-DD')
    for col in ['Submission Date', 'Due Date ']:
        final_submission_df[col] = final_submission_df[col].apply(lambda x: x.strftime("%Y-%m-%d"))

    # submission_df = submission_df.drop(['# of Shortcodes', 'Total Requests per Hitpath', 'Priority Order'], axis=1)


    # Only keep necessary columns
    final_submission_df = final_submission_df[submission_columns]

    # Cannot have NaNs when writing to smartsheet
    final_submission_df = final_submission_df.fillna('')


    # Need to have Hitpath Offer ID column as an int so that Status can be pulled properly
    final_submission_df['Hitpath Offer ID'] = final_submission_df['Hitpath Offer ID'].apply(lambda x: int(x) if x.isdigit() else x)

    return final_submission_df, stats_by_offer

# Make DataFrame, Save as Excel, and Write to Smartsheet

In [42]:
submission_df, stats_by_offer, stats_by_creative, offer_inventory = make_pieces_request_df(request_limit=20)
submission_df

,Hitpath ID,Submission Date,Due Date,Request Type (Request),Creative Approval Status,Offer Type,Hitpath Offer ID,Offer Name,Vertical,Landing Page / Redirect Link,Compliance & Additional Notes (Request),# of Pieces Requested Per Shortcode,# of Pieces Requested Per Toll Free,Shortcode Abbreviations,Requested By,RX Rep,External AM Email,Channel,Platform
0,13059,2024-07-16,2024-07-19,Content,Yes - need advertiser approval,Existing - More Content,13059,EXCLUSIVE - 2024FreeScores,Credit Score,https://www.rentonion.com/rd/r.php?sid=13059&p...,N/A,2,0,FLC\nHZB\nDSS\nMBC\nSVT\nUAA,Script,Mary Beth Howe,Kara Moore kara@cx3ads.com,SC,SS
1,13089,2024-07-16,2024-07-22,Content,Yes - need advertiser approval,Existing - More Content,13089,HomeSafeGuarded Home Insurance - B2D,Home Insurance,https://www.rentonion.com/rd/r.php?sid=13089&p...,N/A,2,0,DSS\nMBC\nFLC\nHZB,Script,Hartley Goode,Madison Housel - madison@btwodirect.com,SC,SS


In [43]:
prev_requests = get_previous_script_requests()

In [44]:
rx_rep_offer_dict = get_rx_rep_offer_counts(submission_df)

In [45]:
from nltk import flatten
from openpyxl.styles import numbers
from openpyxl import Workbook
from openpyxl.styles import Font, Color, colors, fills
from openpyxl.utils.dataframe import dataframe_to_rows

from openpyxl.styles.borders import Border, Side, BORDER_THIN
thin_border = Border(
    left=Side(border_style=BORDER_THIN, color='D3D3D3'),
    right=Side(border_style=BORDER_THIN, color='D3D3D3'),
    top=Side(border_style=BORDER_THIN, color='D3D3D3'),
    bottom=Side(border_style=BORDER_THIN, color='D3D3D3')
)

In [46]:

# Make Recently Tested Content Report
wb = Workbook()
ws = wb.active
all_sheets = []
ws_headers = {}

ws_headers['Content Requests'] = submission_df.columns.tolist()
ws_headers['Stats by Offer'] = stats_by_offer.columns.tolist()
ws_headers['Stats by Creative'] = stats_by_creative.columns.tolist()
ws_headers['Offer Inventory'] = offer_inventory.columns.tolist()
ws_headers['Previous Script Requests'] = prev_requests.columns.tolist()

submission_df_hitpaths = submission_df['Hitpath Offer ID'].tolist()
submission_df_hitpaths = [str(x) for x in submission_df_hitpaths]

stats_by_offer_hitpaths = stats_by_offer['Hitpath Offer ID'].tolist()
stats_by_offer_hitpaths = [str(x) for x in stats_by_offer_hitpaths]


hitpath_requests_removed = [hitpath for hitpath in stats_by_offer_hitpaths if hitpath not in submission_df_hitpaths]


content_request_sheet = wb['Sheet']
content_request_sheet.title = 'Content Requests'

# Column names to format specially
currency_format_cols = ['Revenue', 'eCPM', 'eCPM_min_target']
percent_format_cols = ['OOR', 'CTR', 'CTR_min_target', '30:90 Ratio']
comma_format_cols = ['Optout', 'Clicks', 'Delivered']

for r in dataframe_to_rows(submission_df, index=False, header=True):
    content_request_sheet.append(r)
all_sheets.append(content_request_sheet)

stats_by_offer_sheet = wb.create_sheet('Stats by Offer')
for r in dataframe_to_rows(stats_by_offer, index=False, header=True):
    stats_by_offer_sheet.append(r)
all_sheets.append(stats_by_offer_sheet)

stats_by_creative_sheet = wb.create_sheet('Stats by Creative')
for r in dataframe_to_rows(stats_by_creative, index=False, header=True):
    stats_by_creative_sheet.append(r)
all_sheets.append(stats_by_creative_sheet)

offer_inventory_sheet = wb.create_sheet('Offer Inventory')
for r in dataframe_to_rows(offer_inventory, index=False, header=True):
    offer_inventory_sheet.append(r)
all_sheets.append(offer_inventory_sheet)

prev_requests_sheet = wb.create_sheet('Previous Script Requests')
for r in dataframe_to_rows(prev_requests, index=False, header=True):
    prev_requests_sheet.append(r)
all_sheets.append(prev_requests_sheet)

logic_and_definitions_sheet = wb.create_sheet('Logic and Definitions')
logic_and_definitions_sheet.cell(1, 1).value = 'Logic:'
logic_and_definitions_sheet.cell(1, 2).value = 'https://docs.google.com/document/d/1mYEPQHOprX170u_882O5fiQqLCohWGwmnE34o7iuwuQ/edit?usp=sharing'
logic_and_definitions_sheet.cell(2, 1).value = 'Definitions:'
logic_and_definitions_sheet.cell(2, 2).value = 'https://docs.google.com/document/d/1Og0j8Bwzl8kim9_LCgQ6MLrt2_BkWEiwguCNWsKwQgg/edit?usp=sharing'





for ws in all_sheets:
    dims = {}
    headers = ws_headers[ws.title]

    for row in ws.rows:
        for cell in ws["1:1"]:
            cell.font = Font(bold=True)
        for cell in row:
            newline_count = 1
            if cell.value:
                if type(cell.value)==str:
                    if ('\n' in cell.value):
                        newline_count = cell.value.count('\n')
                dims[cell.column_letter] = max((dims.get(cell.column_letter, 0), len(str(cell.value))))/newline_count 
    for col, value in dims.items():
        ws.column_dimensions[col].width = value

    for col in currency_format_cols:
        if col in headers:    
            col_num = headers.index(col) + 1
            col_letter = convert_number_to_letter(col_num)
            # Format column C as currency to 2 decimal places
            for cell in ws[col_letter]:
                cell.number_format = numbers.FORMAT_CURRENCY_USD_SIMPLE

    for col in percent_format_cols:    
        if col in headers:    
            col_num = headers.index(col) + 1
            col_letter = convert_number_to_letter(col_num)
            # Format column C as currency to 2 decimal places
            for cell in ws[col_letter]:
                cell.number_format = numbers.FORMAT_PERCENTAGE_00

    for col in comma_format_cols:    
        if col in headers:    
            col_num = headers.index(col) + 1
            col_letter = convert_number_to_letter(col_num)
            # Format column C as currency to 2 decimal places
            for cell in ws[col_letter]:
                cell.number_format = '#,##0'

    ws.freeze_panes = 'A2'

# Make entire row orange if offer was removed from requests
for row in stats_by_offer_sheet.iter_rows(min_row=2):
    # If the Sendable column if False, make entire row orange
    if row[ws_headers['Stats by Offer'].index('Hitpath Offer ID')].value in hitpath_requests_removed:
        for cell in row:     
            cell.fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='fce5cd'))
            cell.border = thin_border

# # Make OOR orange if more than 2%
# for row in stats_by_creative_sheet.iter_rows(min_row=2, min_col=ws_headers['Stats by Creative'].index('OOR') + 1, max_col=ws_headers['Stats by Creative'].index('OOR') + 1):
#     value = row[0].value
#     if value is not None and value >= 0.02:
#         stats_by_creative_sheet.cell(row=row[0].row, column=ws_headers['Stats by Creative'].index('OOR') + 1).fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='fce5cd'))

# Pair CTR and eCPM columns with target columns
ctr_comparison = (ws_headers['Stats by Creative'].index('CTR') + 1, ws_headers['Stats by Creative'].index('CTR_min_target') + 1)
eCPM_comparison = (ws_headers['Stats by Creative'].index('eCPM') + 1, ws_headers['Stats by Creative'].index('eCPM_min_target') + 1)

# Make CTR or eCPM orange if less than min target
for stat_comparison in [ctr_comparison, eCPM_comparison]:
    for row in stats_by_creative_sheet.iter_rows(min_row=2):
        value_i = row[stat_comparison[0]-1].value
        value_j = row[stat_comparison[1]-1].value

        if value_i is not None and value_j is not None and value_i < value_j:
            stats_by_creative_sheet.cell(row=row[0].row, column=stat_comparison[0]).fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='fce5cd'))
            stats_by_creative_sheet.cell(row=row[0].row, column=stat_comparison[0]).border = thin_border


# Make entire row orange if not sendable
for row in stats_by_creative_sheet.iter_rows(min_row=2):
    # If the Sendable column if False, make entire row orange
    if not row[ws_headers['Stats by Creative'].index('Sendable')].value:
        for cell in row:     
            cell.fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='fce5cd'))
            cell.border = thin_border

    

# wb.save("Content Request Report - {}.xlsx".format(date.today().strftime("%m_%d_%Y")))

In [58]:
all_sheets

[<Worksheet "Content Requests">,
 <Worksheet "Stats by Offer">,
 <Worksheet "Stats by Creative">,
 <Worksheet "Offer Inventory">,
 <Worksheet "Previous Script Requests">]

In [48]:
import os

today = date.today().strftime("%m_%d_%Y")
save_path = filepath.output_folder + "Content Request Reports/"
filename = os.path.join(save_path, f"Content_Request_Report_{today}.xlsx")

wb.save(filename)

#filename = f"Content_Feedback_Loop_Report_{today}.xlsx"
#wb.save(filename)

In [56]:
# comment by Lili on 7/12
#result = place_in_smartsheet(submission_df, key, sheet_id)

Adding Data to Smartsheet...
Data added to Smartsheet
